## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-28-02-39-28 +0000,nyt,Instagram to Alert Parents to Teens’ Self-Harm...,https://www.nytimes.com/2026/02/27/technology/...
1,2026-02-28-02-34-00 +0000,wsj,Officials at multiple federal agencies have ra...,https://www.wsj.com/politics/national-security...
2,2026-02-28-02-26-38 +0000,nyt,"In a Memoir Draft, Changpeng Zhao of Binance D...",https://www.nytimes.com/2026/02/27/technology/...
3,2026-02-28-02-25-20 +0000,nypost,KTLA insiders detail station bloodbath as belo...,https://nypost.com/2026/02/27/us-news/inside-k...
4,2026-02-28-02-25-02 +0000,nyt,‘We Got to Win the Midterms’: Trump Takes His ...,https://www.nytimes.com/2026/02/27/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2546/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
42,trump,31
104,ai,19
191,warner,17
345,war,17
110,iran,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
67,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...,169
183,2026-02-27-18-25-13 +0000,nypost,Trump makes sobering admission about Iran talk...,https://nypost.com/2026/02/27/us-news/trump-no...,110
28,2026-02-28-00-22-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...,102
187,2026-02-27-18-03-16 +0000,nypost,Forcing Bill Clinton to testify will backfire ...,https://nypost.com/2026/02/27/us-news/forcing-...,94
12,2026-02-28-01-51-56 +0000,nyt,Trump Orders U.S. Agencies to Stop Using Anthr...,https://www.nytimes.com/2026/02/27/us/politics...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
67,169,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...
159,86,2026-02-27-19-26-13 +0000,nypost,Here’s what the massive Paramount-Warner Bros....,https://nypost.com/2026/02/27/business/heres-e...
346,66,2026-02-27-03-31-25 +0000,nypost,JD Vance says ‘no chance’ a US attack on Iran ...,https://nypost.com/2026/02/26/us-news/jd-vance...
270,51,2026-02-27-12-00-00 +0000,wsj,Hybrid cyberattacks have become more common. “...,https://www.wsj.com/articles/new-york-state-el...
150,48,2026-02-27-19-54-00 +0000,wsj,The Pentagon said it shot down a “seemingly th...,https://www.wsj.com/politics/national-security...
28,43,2026-02-28-00-22-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...
314,42,2026-02-27-09-47-19 +0000,bbc,Green Party wins Gorton and Denton by-election...,https://www.bbc.com/news/articles/cr453rvy6kvo...
249,40,2026-02-27-13-19-18 +0000,bbc,Pakistan defence minister says country in 'ope...,https://www.bbc.com/news/articles/cvgjd7pv4y4o...
152,38,2026-02-27-19-52-17 +0000,startribune,"Anderson: Growing up in south Minneapolis, fis...",https://www.startribune.com/dennis-anderson-la...
117,38,2026-02-27-21-35-47 +0000,nypost,World Bank faces calls to fire Israel-bashing ...,https://nypost.com/2026/02/27/business/world-b...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
